In [1]:
import pandas as pd

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
  sys.path.append(module_path)

In [3]:
from constants import SUBMISSION_TIME, START_TIME, END_TIME, JOB_TYPE, SOURCE, SERVER_TYPE, LOCATION, NAMES

In [4]:
INPUT_NAMES = ['task_name', 'instance_num', 'job_name', 'task_type', 'status', START_TIME, END_TIME, 'plan_cpu', 'plan_mem']

In [5]:
t = pd.read_csv('in/batch_task.csv', delimiter=',', header=None, names=INPUT_NAMES)

In [9]:
t['runtime'] = t[END_TIME] - t[START_TIME]

In [12]:
t = t[t['runtime'] >= 0]
t

,task_name,instance_num,job_name,task_type,status,start_time,end_time,plan_cpu,plan_mem,runtime
0,M1,1.0,j_1,1,Terminated,419912,419912,100.0,0.20,0
1,R2_1,1.0,j_2,1,Terminated,87076,87086,50.0,0.20,10
2,M1,1.0,j_2,1,Terminated,87076,87083,50.0,0.20,7
3,R6_3,371.0,j_3,1,Terminated,157297,157325,100.0,0.49,28
4,J4_2_3,1111.0,j_3,1,Terminated,157329,157376,100.0,0.59,47
...,...,...,...,...,...,...,...,...,...,...
14295726,M3,1.0,j_4201003,1,Terminated,149574,149575,100.0,0.30,1
14295727,M2,1.0,j_4201003,1,Terminated,149574,149575,100.0,0.30,1
14295728,M1,1.0,j_4201005,1,Terminated,605611,605611,100.0,0.20,0
14295729,M1,1.0,j_4201007,1,Terminated,171940,171943,50.0,0.30,3


In [13]:
t.groupby(['task_type']).mean()

,instance_num,start_time,end_time,plan_cpu,plan_mem,runtime
task_type,,,,,,
1,110.640126,387247.572324,387313.512692,79.571660,0.331708,65.940368
2,19.083333,665313.750000,665459.916667,15.000000,0.043333,146.166667
3,204.405713,365535.146268,366220.245416,113.141282,0.660146,685.099148
4,7.888466,377697.314654,377767.489119,50.000000,0.190000,70.174465
5,1.000000,368484.830988,368907.078187,390.936286,1.761954,422.247199
6,12.813705,449808.026995,449945.834150,14.987233,0.043533,137.807155
8,192.709377,389652.916037,390146.961601,148.294027,0.558111,494.045565
9,90.606297,376767.309338,377028.717714,97.548525,0.495575,261.408375
10,9.337966,417223.659847,417436.300284,89.617943,0.100000,212.640436


In [14]:
def task_type_to_job_type(task_type: int) -> int:
  # short
  if task_type == 1 or task_type == 4:
    return 1
  # medium
  elif task_type == 2 or task_type == 6 or task_type == 9 or task_type == 10 or task_type == 12:
    return 2
  # long
  elif task_type == 3 or task_type == 5 or task_type == 8:
    return 3
  # very long
  elif task_type == 11:
    return 4

In [15]:
t[SUBMISSION_TIME] = t[START_TIME]
t[JOB_TYPE] = t['task_type'].apply(task_type_to_job_type)
t[SOURCE] = 0
t[SERVER_TYPE] = 0
t[LOCATION] = 0

/home/jonhue/projects/jonhue/bachelors-thesis/venv/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [16]:
t = t.sort_values(by=SUBMISSION_TIME)

In [20]:
t[NAMES].to_csv('out/trace.csv', index=False)